In [52]:
# Import libs
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from google.cloud import storage

In [58]:
# GCS details
gcs= pa.fs.GcsFileSystem()
bucket_name = 'de-zoomcamp-dominik-1'
project_id = "dtc-de-course-412911"
table_name = "green_trips_data"
root_path = f'{bucket_name}/{table_name}'

In [60]:
years =  [2020]
months = range(1,13)
for year in years:
    for month in months: 
        taxi_dtypes = {
                            'VendorID': pd.Int64Dtype(),
                            'store_and_fwd_flag': str,
                            'RatecodeID': pd.Int64Dtype(),
                            'PULocationID': pd.Int64Dtype(),
                            'DOLocationID': pd.Int64Dtype(),
                            'passenger_count': pd.Int64Dtype(),
                            'trip_distance': float,
                            'fare_amount': float,
                            'extra': float,
                            'mta_tax': float,
                            'tip_amount': float,
                            'tolls_amount': float,
                            'ehail_fee': float,
                            'improvement_surcharge': float,
                            'total_amount': float,
                            'payment_type': pd.Int64Dtype(),
                            'trip_type': pd.Int64Dtype(),
                            'congestion_surcharge': float
                        }    
        parse_dates = ['lpep_pickup_datetime', 'lpep_dropoff_datetime']
        url = f'https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_{year}-{month:02}.csv.gz'
        df = pd.read_csv(url, dtype=taxi_dtypes, sep=',', compression='gzip', parse_dates=parse_dates, low_memory=False)
        
        # Column for partitioning by pickup time
        df['lpep_pickup_date'] = df['lpep_pickup_datetime'].dt.strftime("%Y%m%d")
        df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
        df['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime'].dt.strftime("%Y-%m-%d %H:%M:%S")
        
        # Write do GCS Bucket
        table = pa.Table.from_pandas(df)
        pq.write_to_dataset(
                table,
                root_path = root_path,
                partition_cols = ['lpep_pickup_date'],
                filesystem=gcs)
        print(f'{month} - {year} added go GCS!')

1 - 2020 added go GCS!
2 - 2020 added go GCS!
3 - 2020 added go GCS!
4 - 2020 added go GCS!
5 - 2020 added go GCS!
6 - 2020 added go GCS!
7 - 2020 added go GCS!
8 - 2020 added go GCS!
9 - 2020 added go GCS!
10 - 2020 added go GCS!
11 - 2020 added go GCS!
12 - 2020 added go GCS!


In [ ]:
df